<h1>Ejercicio 1: IO y las imágenes</h1>
    En el siguiente ejercicio, se te entrega el archivo imagen.bmp. Tu deber es "encriptarla", para volver no legible. 
    Posteriormente, deberás crear "la llave", que te permita devolver la imagen a su estado original. Para esto deberás seguir los siguientes pasos:
    
1. Para encriptar, deberás:
    - Leer la imagen como bytes
    - Ir guardando los chunks en un bytearray
    - Deberás procesar la imagen en chunks de 10 bytes
    - A cada uno de esos chunks, deberás agregarle ceros hasta llegar a los 20 bytes
    - Agregar al bytearray creado para tu archivo encriptado los chunks rellenados.
    - Escribir el nuevo archivo con el nombre "imagen_virus.jpg"

2. Para crear la función llave que desencripte el archivo:
    - Crear un bytearray para los bytes recuperados.
    - Leer el archivo como bytes
    - Iterar sobrecada uno de los chunks (Recuerda que los nuevos bytes están organizados de a 20)
    - Recuperar los bytes originales.
    - Juntar los bytes recuperados en tu nuevo bytearray
    - Escribir el nuevo archivo con el nombre de "imagen_recuperada.jpg"
        
   


        
    

In [ ]:
def transforma_a_byter(path_imagen):
    with open(path_imagen, "rb") as archivo:
        imagen = archivo.read()
        archivo_sub_10 = bytearray()
        for pedacitos in range(0, len(imagen), 10):
            chunk = imagen[pedacitos:pedacitos + 10] # Va de a 10
            print(len(chunk))
            trozo_rellenado = chunk.zfill(20)
             #Agrego 10 caracteres
            print(len(trozo_rellenado))
            print("----------------Terminó un chunk-------------------")
            archivo_sub_10 += trozo_rellenado
        with open("imagen_virus.jpg", "wb") as archivo_malo:
            archivo_malo.write(archivo_sub_10)

def quita_virus(path_imagen):
    bytes_recuperados = bytearray()
    with open(path_imagen, "rb") as archivo_malo:
        imagen_mala = archivo_malo.read()
        for pedacitos in range(0, len(imagen_mala)-40, 20):
            chunk = imagen_mala[pedacitos+10:pedacitos+20]
            bytes_recuperados += chunk
        for pedacitos in range(len(imagen_mala)-40, len(imagen_mala)+40, 20):
            chunk = imagen_mala[pedacitos + 11:pedacitos + 20]
            bytes_recuperados += chunk
        with open("imagen_recuperada.jpg", "wb") as archivo_recuperado:
            archivo_recuperado.write(bytes_recuperados)
            

transforma_a_byter("imagen.bmp")
quita_virus("imagen_virus.jpg")            

<h1>Ejercicio 2: Serialización usando Pickle y JSON</h1>

Eres conocid@ por tod@s por tu gran amor al cine, razón por la cual tu novia Victoria te ha pedido que por favor le recomiendes películas buenas para poder ver. Para hacer esta recomendación, debes hacer uso del archivo `movies.json`. 
Este archivo tiene un diccionario por película con varias características, de las cuales solamente debes guardar algunas. El procedimiento que debes llevar a cabo es el siguiente:

1. Completar la funcion `cargar_películas()` que debe deserializar la información de las películas del archivo `movies.json`, solamente incluyendo los datos que son relevantes especificados en el archivo `caracteristicas.pkl`, mediante la funcion `hook_peliculas()`. Esta última debe retorna una lista de objetos de la clase Pelicula. Notar que el archivo `caracteristicas.pkl` también debe ser deserializado, utilizando Pickle, para obtener la información en el.
2. Completar la función `recomendar_peliculas(lista_peliculas)` que recibe una lista de objetos de la clase Pelicula. Esta debe ser capaz de serializar, mediante Pickle, cada película en un archivo llamado `recomendaciones.pkl`. Para la serialización, debes completar el método `__getstate__()` para poder agregar una opinión (tu recomendación) al diccionario a serializar de cada película. Notar que los strings de opinión a elegir se encuentran en el archivo `opiniones.pkl`, por lo que debes deserializar este también, mediante Pickle, para obtenerlas. Puedes usar la librería `random` importada para elegir una opinión al azar.








In [4]:
import json
import pickle
import random

 
class Pelicula:

    def __init__(self, title=None, category_name=None, release_year=None, running_time=None):

        self.title = title
        self.category_name = category_name
        self.release_year = release_year
        self.running_time = running_time

    def __getstate__(self):
        """
        Aquí debes personalizar la serialización de las películas, agregando una opinion al 
        diccionario. Recuerda obtener las opiniones deserializando el archivo opiniones.pkl 
        con Pickle.
        """
        with open("opiniones.pkl", "rb") as file:
            opiniones = pickle.load(file)

        opinion = random.choice(opiniones)
        new = self.__dict__.copy()
        new.update({"opinion": opinion})

        return new


def hook_peliculas(peliculas):
    """
    Debes utilizar esta función para personalizar la deserialización de las películas en 
    movies.json.
    """
    with open("caracteristicas.pkl", "rb") as file:
        caracteristicas = pickle.load(file)
    filtrado = {c: peliculas[c] for c in caracteristicas}

    return Pelicula(**filtrado)


def cargar_peliculas():
    """
    Esta función debe deserializar la información de cada película, pero icluyendo solamente la
    información especificada en caracteristicas.pkl (este archivo debe ser deserializado con Pickle) 
    utilizando la función hook_peliculas() para el filtrado.
    """
    with open("movies.json", encoding="utf-8") as file:
        return json.load(file, object_hook=hook_peliculas)


def recomendar_peliculas(peliculas):
    """
    Se deben serializar las películas en el archivo caracteristicas.pkl utilizando Pickle, 
    incluyendo un mensaje de opinión mediante el método __getstate__() de la clase Pelicula.
    """
    with open("recomendacion.pkl", "wb") as file:
        for p in peliculas:
            pickle.dump(p, file)
            

In [5]:
# Main code
"""
Al correr este código, deberías obtener un archivo llamado recomendacion.pkl con la información
de cada película y la opinion para cada una de estas.
"""
peliculas = cargar_peliculas()
recomendar_peliculas(peliculas)

In [6]:
"""
El siguiente código puede ser corrido para comprobar que se realizo el ejercicio correctamente.
Deberían imprimirse diccionarios por película, con todas las caracteristicas de cada película del 
archivo caracteristicas.pkl como keys y una key "opinion" que tiene como value alguna de las tres 
opiniones del archivo opiniones.pkl.
"""

with open("recomendacion.pkl", "rb") as file:
    peliculas_recomendadas = list()
    while True:
        try:
            peliculas_recomendadas.append(pickle.load(file))
        except EOFError:
            break
for p in peliculas_recomendadas:
    print(p.__dict__)


{'title': 'Akira', 'category_name': 'Science Fiction', 'release_year': 1987, 'running_time': 124, 'opinion': 'Esta pelicula es buena, dentro de las mejores.'}
{'title': 'Alien', 'category_name': 'Science Fiction', 'release_year': 1979, 'running_time': 116, 'opinion': 'Esta pelicula es buena pero no es nada del otro mundo.'}
{'title': 'Aliens', 'category_name': 'Science Fiction', 'release_year': 1986, 'running_time': 137, 'opinion': 'Esta pelicula es buena, dentro de las mejores.'}
{'title': 'Avatar', 'category_name': 'Science Fiction', 'release_year': 2009, 'running_time': 178, 'opinion': 'Esta pelicula es buena, dentro de las mejores.'}
{'title': 'Blade Runner', 'category_name': 'Science Fiction', 'release_year': 1992, 'running_time': 117, 'opinion': 'Esta pelicula me encanto, recomiendo fuertemente que la veas si o si.'}
{'title': 'Chappie', 'category_name': 'Science Fiction', 'release_year': 2015, 'running_time': 120, 'opinion': 'Esta pelicula me encanto, recomiendo fuertemente que 

<h1> Ejercicio 3: Serialización usando JSON </h1>

Durante las caóticas inscripciones de cursos para el segundo semestre del 2020 un hacker aprovechó la situación y corrompió la base de datos de los estudiantes, dejándolos con asignaturas inexistentes en el sistema. Afortunadamente, existe un archivo `asignaturas.json` en donde están los cursos disponibles. Tu trabajo será instanciar los Alumnos con los cursos correctos, filtrando (por medio de un hook) aquellos cursos que no correspondan.

In [6]:
import json


class Alumno:
    def __init__(self, nombre, num_alumno, cursos):
        self.nombre = nombre
        self.num_alumno = num_alumno
        self.cursos = cursos

    def __repr__(self):
        string_cursos = ', '.join(self.cursos)
        return f'Asignaturas de {self.nombre}: \n{string_cursos}\n'


with open('asignaturas.json') as file:
    asignaturas = json.load(file)


def hook_function(diccionario):
    cursos_validos = list(set(diccionario['cursos']) & set(asignaturas))
    diccionario.update({'cursos': cursos_validos})
    return Alumno(**diccionario)


with open('alumnos.json') as file:
    lista_alumnos = json.load(file, object_hook=hook_function)

print(*lista_alumnos, sep='\n')


Asignaturas de Lucas: 
Bases de datos, Analisis funcional, Calculo 2

Asignaturas de Pedro: 
Programacion avanzada, Analisis funcional, Bases de datos

Asignaturas de Sebastián: 
Programacion avanzada, Bases de datos, El Mundo de los medicamentos, Analisis de regresion, Jesus distintas comprensiones

Asignaturas de Alejandra: 
Bases de datos, Termodinamica, Innovacion, Optimizacion

Asignaturas de Pedro: 
Matematicas discretas, El Mundo de los medicamentos, Intro a la programacion

Asignaturas de Sofía: 
Electricidad y magnetismo, Bases de datos, El Mundo de los medicamentos, Analisis de regresion, Jesus distintas comprensiones

Asignaturas de Camila: 
Bases de datos, Optimizacion

Asignaturas de Nicolás: 
Programacion avanzada, Bases de datos, Termodinamica, El Mundo de los medicamentos, Analisis de regresion

Asignaturas de Luis: 
Bases de datos, Matematicas discretas, Analisis de regresion, Jesus distintas comprensiones, Calculo 2

Asignaturas de Florencia: 
Innovacion, Electricidad